Let us first generate a website without database integration using Gemini.

In [1]:
import google.generativeai as genai

import yaml
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)
GEMINI_API_KEY = config['GEMINI_API_KEY']
genai.configure(api_key=GEMINI_API_KEY)

/Users/anupamsobti/anaconda3/envs/webdevai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
import os

def save_file(filename: str, content: str, root_dir: str = 'site') -> str: #save file with backup
    """
    Saves raw text in the content string to a file specified by filename string
    """
    if not os.path.exists(root_dir):
        os.makedirs(root_dir)
    filepath = os.path.join(root_dir, filename)
    if os.path.exists(filepath):
        os.rename(filepath, filepath + '.1')
    with open(filepath, 'w') as f:
        f.write(content)

In [3]:
current_task_prompt = "<task> Generate a website that displays data from a few courses and their lectures. The lectures should be in a table with lecture number and description of each lecture. </task>"

general_instructions = \
"""<instructions>
1. Keep style consistent. Use tailwind classes for styles and green/white theme in the website.
2. Keep elements responsive wherever possible.
3. Generate separate files wherever possible and save them using the save_file tool.
</instructions>"""

system_prompt = "You are expert web designer who understands various frameworks such as nextjs, react as well as hardcoded solutions. You follow best practices wherever possible such as SOLID principles. You have the taste to prefer simple things. You use Python and FastAPI as primary backend functionality."

about_the_project = "<about>The website being developed is a demonstrator for concepts in class. In this particular case, I will be demonstrating how to fetch data from a db and display it on the frontend.</about>"

In [6]:
model = genai.GenerativeModel('gemini-2.0-flash-exp',
                              system_instruction=system_prompt, 
                              tools=[save_file])

# Open a new chat
chat = model.start_chat(enable_automatic_function_calling=True)
chat.send_message("\n".join([general_instructions, about_the_project, current_task_prompt]))

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The `index.html` file sets up the basic HTML structure, including the link to Tailwind CSS and the `script.js` file. It also includes a `div` with the id \"root\" where the content will be rendered, as well as a heading.\n\nThe `script.js` file contains the JavaScript code to fetch the course data and display it in a structured format. It iterates through the courses and their lectures, creating HTML elements dynamically and appending them to the \"courses\" div. The displayCourses function takes the courses data as input and generates the HTML for each course, including a table for the lectures. The table includes lecture number and description.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
   

To edit an existing website, more context is required. 

- What are the current files?
- What exactly do you want to edit?

In [19]:
def get_site_context(site_dir: str = "site"):
    site_context = ""
    for filename in os.listdir(site_dir):
        filepath = os.path.join('site', filename)
    if os.path.isfile(filepath):
        with open(filepath, 'r') as f:
            file_content = f.read()
        site_context += f'\n<file name="{filename}">\n{file_content}\n</file>\n'
    return site_context


def get_edit_prompt(new_prompt : str = ""):
    file_context = get_site_context('site')
    return "\n".join([about_the_project, general_instructions, file_context, new_prompt])

edit_prompt = get_edit_prompt("<task>make the color blue and white instead of green and white</task>")

In [17]:
chat = model.start_chat(enable_automatic_function_calling=True)
chat.send_message(edit_prompt)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "I have updated the script to use a blue and white color scheme instead of green and white. The changes include updating the text and background colors in the `courseNameElement` and `tableHead` elements.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.09544274973314862
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 1382,
        "candidates_token_count": 43,
        "total_token_count": 1425
      }
    }),
)

Now, before we start using the DB, we need to tell the LLM about its structure. So, I independently looked at chatGPT for code on how to get the schema info ready for an LLM from an sqlite DB. Here's the code:


In [18]:
import sqlite3
import json

dbname = 'course_website.db'
output_file = 'db_schema.json'

# Connect to the database and extract schema
with sqlite3.connect(dbname) as con:
    cursor = con.cursor()
    cursor.execute("SELECT name, type, sql FROM sqlite_master WHERE type IN ('table', 'index', 'view')")
    
    schema_info = []
    for row in cursor.fetchall():
        schema_info.append({
            "name": row[0],  # Table, Index, or View name
            "type": row[1],  # 'table', 'index', or 'view'
            "sql": row[2]    # SQL statement to create the object
        })

    cursor.close()

# Convert schema info to JSON format
with open(output_file, 'w') as f:
    json.dump(schema_info, f, indent=4)

print(f"Schema exported to {output_file}")


Schema exported to db_schema.json


In [26]:
db_prompt = f"""
                <schema>
                {schema_info}
                </schema>
            """
edit_prompt = get_edit_prompt(f"""
                              I am providing you with a DB schema with information on tables, and the exact sql command that created this table. 
                              {db_prompt}
                              
                              <task>
                              Create a FastAPI route to get lecture information and course information from the Database. Save this into files using the save_file tool.
                              </task>""")

model = genai.GenerativeModel('gemini-2.0-flash-exp',
                              system_instruction=system_prompt,
                              tools=[save_file],
                              generation_config={
                                  'temperature': 0.7,  # Adjust this value between 0.0 and 1.0
                                  # You can also add other generation parameters like:
                                  # 'top_p': 0.8,
                                  # 'top_k': 40,
                                  # 'max_output_tokens': 2048,
                              })

chat = model.start_chat(enable_automatic_function_calling=True)
chat.send_message(edit_prompt)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "I have created the FastAPI application and saved it to `main.py`. The application defines two routes: `/courses/` to retrieve a list of courses and `/lectures/` to retrieve a list of lectures. Both routes return a list of dictionaries containing the relevant information.  A SQLite database `test.db` is used. Remember to populate the database with data for the API to return meaningful results.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.18697782404282515
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 1639,
        "candidates_token_count": 85,
        "total_token_count": 1724
      }
    }),
)

In [ ]:
! fastapi dev main.py

All right! Now, our API endpoints are up and running. Now, we need to edit our website to incorporate these endpoints.

In [28]:
edit_prompt = get_edit_prompt( \
                              f"""<info> 
                              My database schema information from course_website.db is present in {db_prompt}.
                              </info>
                              <task> Modify the website files to use the API endpoints running on http://120.0.0.1:8000 to fetch the list of courses and lectures.
                              </task>""")

chat = model.start_chat(enable_automatic_function_calling=True)
chat.send_message(edit_prompt)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "I have successfully updated the `script.js` file to fetch course and lecture data from the specified API endpoints and dynamically render it in the browser. The updated script also includes error handling and uses Tailwind CSS classes for styling.\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.2189634607193318
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 2824,
        "candidates_token_count": 47,
        "total_token_count": 2871
      }
    }),
)

Now, try to modify the main.py that you generate for the fastapi endpoints to also serve the static files.